##<8-4 파인튜닝 마친 모델로 문장 생성하기>

이번 절에서는 파인튜닝을 마친 문장 생성 모델을 인퍼런스하는 과정을 실습해 보겠습니다.  
이번 실습에서는 SK텔레콤이 공개한 KoGPT2 모델을 NSMC로 파인튜닝한 모델을 인퍼런스합니다.

컨텍스트를 받아 컨텍스트 뒤에 이어지는 문장을 생성해 반환하는 웹 서비스인데요, 컨텍스트를 토큰화하고 이를 모델에 넣어 다음 토큰 등장 확률을 계산한 뒤 다음 토큰을 뽑습니다.  
이어 기존 컨텍스트에 이번에 뽑은 다음 토큰을 이어 붙여 새로운 컨텍스트를 만들고 이를 모델에 입력해 다음 토큰을 샘플링합니다. 이를 정해진 길이가 될 때까지 반복하고 후처리를 통해 컨텍스트를 포함한 토큰 인덱스 시퀀스를 문자열로 변환해 응답하게 만드는 방식입니다.

<center><그림 - 문장 생성 웹 서비스></center>

<p align="center"><img src="https://i.imgur.com/I4lGm3J.jpg">


### 문장 생성 웹 서비스 만들기
---
<**1단계**> **코랩 노트북 초기화하기**  

코랩에서 [`내 드라이브에 복사`]를 진행하고 [런타임 $\rightarrow$ `런타임 유형 변경`] 메뉴에서 하드웨어 가속을 사용하지 않도록 [None]을 선택합니다.

<**2단계**> **환경 설정하기**

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 755 kB/s 
     |████████████████████████████████| 57 kB 4.6 MB/s 
     |████████████████████████████████| 806 kB 27.7 MB/s 
     |████████████████████████████████| 2.8 MB 40.7 MB/s 
     |████████████████████████████████| 636 kB 41.5 MB/s 
     |████████████████████████████████| 408 kB 29.4 MB/s 
     |████████████████████████████████| 829 kB 28.7 MB/s 
     |████████████████████████████████| 136 kB 53.3 MB/s 
     |████████████████████████████████| 895 kB 45.8 MB/s 
     |████████████████████████████████| 3.3 MB 20.0 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 33.4 MB/s 
     |████████████████████████████████| 96 kB 5.0 MB/s 
     |████████████████████████████████| 144 kB 49.9 MB/s 
     |████████████████████████████████| 271 kB 56.2 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


이번 실습에서는 SKT가 공개한 KoGPT2 모델을 NSMC로 파인튜닝한 모델을 인퍼런스합니다.  
다음 코드를 실행하면 관련 설정을 할 수 있습니다.

In [3]:
!pip install torch

In [4]:
#!pip install torchmetrics

<**3단계**> **토크나이저 및 모델 불러오기**  

다음 코드를 실행해 토크나이저와 모델을 불러옵니다.

In [6]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
import pytorch_lightning as pl
print(pl.__version__)

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-gedflla4
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-gedflla4
  Running command git submodule update --init --recursive -q
  From https://github.com/PyTorchLightning/lightning-tutorials
   * branch            290fb466de1fcc2ac6025f74b56906592911e856 -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pytorch-lightning: filename=pytorch_lightning-1.7.0.dev0-py3-none-any.whl size=573921 sha256=2ae49770a0eb8c5dae07a1556cc2b0bc8f8ca3cc2aa15448373c5897afc23259
  Stored in directory: /tmp/pip-ephem-wheel-cache-s7df8sv8/wheels/18/24/dd/8e13b7dfcda990eb2b099a57bb6d705f6d43fb53ecb4ed07bf
Successfully built pytorch-lightning
  Attempting uninstall: pyDeprecate
    Found existing installation: pyDeprecate 0.3.0
    Uninstalling pyDeprecate-0.3.0:


1.3.4


In [7]:
#인퍼런스 설정
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-generation/epoch=1-val_loss=2.29.ckpt


## * 오류에 관해서 솔루션을 찾아서 다시 업로드 하겠습니다!
### - 직접 파이토치 라이트닝 깃허브와 스택오버플로우를 통해 오류를 파악하여 해결했습니다.  

 https://github.com/PyTorchLightning/pytorch-lightning/issues/6210   

 https://stackoverflow.com/questions/66538407/unable-to-import-pytorch-lightning-on-google-colab
 

In [8]:
#토크나이저 초기화
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [9]:
#모델 불러오기
import torch
from transformers import GPT2Config, GPT2LMHeadModel
pretrained_model_config = GPT2Config.from_pretrained(
    args.pretrained_model_name,
)
model = GPT2LMHeadModel(pretrained_model_config)
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


<**4단계**> **모델 출력값 만들고 후처리하기**  

다음 코드는 인퍼런스 과정을 정의한 함수입니다.   
우선 문장(`prompt`)을 입력받아 토큰화하고 인덱싱한 뒤 파이토치 텐서로 만듭니다(`input_ids`). 이를 모델에 넣어 이후 입력 문장에 이어지는 토큰 인덱스 시퀀스(`generated_ids`)를 생성합니다.  
마지막으로 토큰 인덱스 시퀀스를 사람이 보기 좋은 형태의 문장으로 변환해 반환합니다.  
한편 `top_p`, `top_k` 등은 앞 절에서 다뤘던 것처럼 확률값이 낮은 토큰을 제외하는 샘플링 관련 인자입니다.

In [10]:
#인퍼런스
def inference_fn(
        prompt,
        min_length=10,
        max_length=20,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

<**5단계**> 웹 서비스 시작하기   

앞 단계에서 정의한 인퍼런스 함수 `inferece_fn()`을 가지고 다음 코드를 실행하면 파이썬 플라스크를 활용한 웹 서비스를 띄울 수 있습니다.

In [11]:
!mkdir /root/.ngrok2 && echo "authtoken: {279c6rgTRIX2kOe4VIaoCJXy0u8_2fAuAWUJwnY1Q22wwoqra}" > /root/.ngrok2/ngrok.yml

In [12]:
#웹 서비스
from ratsnlp.nlpbook.generation import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.generation.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.7/http/client.py",

이제 인퍼런스 준비가 모두 끝났습니다. 웹 브라우저에 문장 생성 화면이 열리면 다음처럼 각종 설정값을 지정하고 프롬프트에 `긍정 아 정말`을 입력합니다. 그러면 모델이 나머지 문장을 생성해서 보여 줍니다. <8-2>절에서 NSMC 데이터를 '레이블(긍정 혹은 부정) + 리뷰 문장' 형태로 가공해 파인튜닝했으므로 `긍정 아 정말` 뒤에 생성된 문장의 극성은 긍정임을 확인할 수 있습니다.  


그리고 더 아래의 그림은 설정값은 같으나 프롬프트(입력 문장)만 다르게 입력한 결과입니다. 부정 아 정말 뒤에 생성된 문장의 극성은 부정임을 알 수 있습니다.

<center><그림 - 문장 생성 예></center>

<p align="center"><img src="https://i.imgur.com/KpAWR5z.png">


<p align="center"><img src="https://i.imgur.com/FW1nAgj.png">


### 맺음말
---
이번 장에서는 문장 생성 모델을 살펴봤습니다.  
문장 생성 모델은 문서 분류, 문장 쌍 분류, 개체명 인식, 질의응답 등 BERT 계열 과제와 달리 모델 파인튜닝만큼이나 샘플링 전략이 중요합니다. 똑같은 모델이라도 샘플링 전략을 어떻게 수립하느냐에 따라 생성되는 문장의 내용과 품질이 확연하게 달라집니다.

OpenAI GPT-3 등 문장 생성에 쓰이는 트랜스포머 계열 언어 모델의 크기가 기하급수로 커지고 있고, 그 성능 또한 눈부시게 발전하고 있습니다. 모델 크기가 일정 수준보다 큰 트랜스포머 계열 언어 모델은 모델 입력(컨텍스트 혹은 프롬프트)을 섬세하게 구성하면 기계 독해, 분류, 번역 등 다양한 과제를 수행할 수 있는 것으로 알려져 있습니다. 최근의 자연어 처리는 이처럼 트랜스포머 계열 언어 모델 덕분에 발전을 거듭하고 있습니다.